<a href="https://colab.research.google.com/github/iragonz/Asistente-tur-stico-Tenerife/blob/main/Asistente_tur%C3%ADstico_tenerife_Iv%C3%A1n_Ramos_Gonz%C3%A1lez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Instalar las dependencias (VERSIÓN COMPLETA)
!pip install -q langchain langchain-google-genai langchain-community pypdf faiss-cpu python-dotenv langchain-text-splitters

print("✅ Todas las dependencias instaladas correctamente")

✅ Todas las dependencias instaladas correctamente


In [13]:
# Imports DEFINITIVOS (sin imports problemáticos)
import os
from datetime import datetime
import random

# Google Generative AI
import google.generativeai as genai

# LangChain básico
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

# LangChain Google GenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

#API KEY
GOOGLE_API_KEY = "tu API key aquí"

# Configurar API
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

print("Imports realizados correctamente")
print("Configuración completada")
print(f"Fecha actual: {datetime.now().strftime('%Y-%m-%d')}")

Imports realizados correctamente
Configuración completada
Fecha actual: 2026-01-12


In [14]:
# Función get_weather
def get_weather(fecha: str, location: str = "Tenerife") -> dict:
    """
    Simula la obtención del clima para una fecha específica en Tenerife.

    Args:
        fecha: Fecha en formato YYYY-MM-DD
        location: Ubicación (por defecto Tenerife)

    Returns:
        dict con información del clima o error
    """
    try:
        # Validar formato de fecha
        datetime.strptime(fecha, '%Y-%m-%d')

        # Log de intento de llamada
        print(f"Llamando a get_weather(fecha='{fecha}', location='{location}')")

        # Simulación de datos climáticos realistas de Tenerife
        zonas = {
            "norte": {
                "temp": (18, 24),
                "descripcion": "Nublado parcial con posibles chubascos",
                "humedad": (70, 90)
            },
            "sur": {
                "temp": (22, 28),
                "descripcion": "Soleado y despejado",
                "humedad": (50, 70)
            },
            "teide": {
                "temp": (5, 15),
                "descripcion": "Frío y despejado, posible viento fuerte",
                "humedad": (30, 50)
            }
        }

        # Determinar zona según location
        zona_key = "sur"  # Por defecto
        if any(x in location.lower() for x in ["norte", "la laguna", "puerto", "santa cruz", "anaga"]):
            zona_key = "norte"
        elif "teide" in location.lower():
            zona_key = "teide"

        zona = zonas[zona_key]
        temp_min, temp_max = zona["temp"]
        hum_min, hum_max = zona["humedad"]

        resultado = {
            "fecha": fecha,
            "ubicacion": location,
            "zona": zona_key,
            "temperatura": f"{random.randint(temp_min, temp_max)}°C",
            "descripcion": zona["descripcion"],
            "humedad": f"{random.randint(hum_min, hum_max)}%",
            "viento": f"{random.randint(10, 30)} km/h",
            "probabilidad_lluvia": f"{random.randint(5, 30)}%" if zona_key == "norte" else f"{random.randint(0, 10)}%",
            "recomendacion": "Ideal para playa" if zona_key == "sur" else "Lleva paraguas" if zona_key == "norte" else "Abrígate bien",
            "status": "success"
        }

        print(f"Clima obtenido exitosamente")
        return resultado

    except ValueError as e:
        error_msg = f"Formato de fecha inválido. Use YYYY-MM-DD. Error: {str(e)}"
        print(f"{error_msg}")
        return {
            "status": "error",
            "error": error_msg,
            "fecha_solicitada": fecha
        }
    except Exception as e:
        error_msg = f"Error inesperado: {str(e)}"
        print(f"{error_msg}")
        return {
            "status": "error",
            "error": error_msg
        }

# Pruebas de la función
print("\n" + "="*60)
print("PRUEBAS DE get_weather")
print("="*60)

# Prueba 1: Sur de Tenerife
print("\n1. Sur de Tenerife:")
print(get_weather("2024-02-15", "Playa de las Américas"))

# Prueba 2: Norte de Tenerife
print("\n2. Norte de Tenerife:")
print(get_weather("2024-02-15", "Puerto de la Cruz"))

# Prueba 3: El Teide
print("\n3. El Teide:")
print(get_weather("2024-02-15", "Parque Nacional del Teide"))

# Prueba 4: Error de formato
print("\n4. Prueba de error:")
print(get_weather("fecha-invalida", "Tenerife"))

print("\n" + "="*60)


PRUEBAS DE get_weather

1. Sur de Tenerife:
Llamando a get_weather(fecha='2024-02-15', location='Playa de las Américas')
Clima obtenido exitosamente
{'fecha': '2024-02-15', 'ubicacion': 'Playa de las Américas', 'zona': 'sur', 'temperatura': '28°C', 'descripcion': 'Soleado y despejado', 'humedad': '60%', 'viento': '27 km/h', 'probabilidad_lluvia': '7%', 'recomendacion': 'Ideal para playa', 'status': 'success'}

2. Norte de Tenerife:
Llamando a get_weather(fecha='2024-02-15', location='Puerto de la Cruz')
Clima obtenido exitosamente
{'fecha': '2024-02-15', 'ubicacion': 'Puerto de la Cruz', 'zona': 'norte', 'temperatura': '24°C', 'descripcion': 'Nublado parcial con posibles chubascos', 'humedad': '78%', 'viento': '26 km/h', 'probabilidad_lluvia': '26%', 'recomendacion': 'Lleva paraguas', 'status': 'success'}

3. El Teide:
Llamando a get_weather(fecha='2024-02-15', location='Parque Nacional del Teide')
Clima obtenido exitosamente
{'fecha': '2024-02-15', 'ubicacion': 'Parque Nacional del T

In [15]:
# Cargar y procesar la guía turística
print("Cargando guía turística de Tenerife...")

# Verificar que el PDF existe
import os
pdf_path = "data/TENERIFE.pdf"

if not os.path.exists(pdf_path):
    print(f"ERROR: No se encuentra el archivo {pdf_path}")
    print("Por favor, sube el archivo TENERIFE.pdf a la carpeta 'data'")
    print("\nPasos:")
    print("1. Haz clic en el icono de carpeta a la izquierda")
    print("2. Crea una carpeta llamada 'data'")
    print("3. Sube el archivo TENERIFE.pdf ahí")
else:
    print(f"Archivo encontrado: {pdf_path}")

    # Cargar PDF
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    print(f"Documento cargado: {len(documents)} páginas")

    # Dividir en chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks = text_splitter.split_documents(documents)
    print(f"Documento dividido en {len(chunks)} chunks")

    # Mostrar estadísticas
    print(f"\nEstadísticas:")
    print(f"   - Total de caracteres: {sum(len(doc.page_content) for doc in documents):,}")
    print(f"   - Promedio por chunk: {sum(len(chunk.page_content) for chunk in chunks) // len(chunks)} caracteres")

    # Mostrar ejemplo de un chunk
    print("\n--- Ejemplo de chunk ---")
    print(chunks[0].page_content[:400] + "...")

Cargando guía turística de Tenerife...
Archivo encontrado: data/TENERIFE.pdf
Documento cargado: 25 páginas
Documento dividido en 30 chunks

Estadísticas:
   - Total de caracteres: 16,094
   - Promedio por chunk: 570 caracteres

--- Ejemplo de chunk ---
TENERIFE – LUGARES DE INTERÉS 
SITIOS QUE VER 
 
ZONA NORTE 
 
• Santa Cruz de Tenerife: 
Santa Cruz de Tenerife es la capital de la isla. Quizás la ruta a seguir si vais a Santa 
Cruz sería: 
- Aparcar en el aparcamiento del Parque Marítimo (ubicación). 
- Caminar por la Avenida Marítima hasta Plaza de España (ubicación). 
- Por el camino de la Avenida Marítima, ver el auditorio de Tenerife (ubic...


In [16]:
# Crear embeddings y vector store
print("Instalando modelo de embeddings...")

# Instalar sentence-transformers si no está
!pip install -q sentence-transformers

from langchain_community.embeddings import HuggingFaceEmbeddings

print("Creando embeddings con modelo local...")

# Usar modelo local de embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Modelo de embeddings cargado (funciona offline y sin cuotas)")

print("Creando vector store con FAISS...")
# Crear vector store FAISS
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

print("Vector store creado con éxito")
print(f"   - Total de vectores: {len(chunks)}")

# Probar búsqueda semántica
print("\n" + "="*60)
print("PRUEBA DE BÚSQUEDA SEMÁNTICA")
print("="*60)

query = "¿Qué playas hay en el norte de Tenerife?"
print(f"\n Query: {query}")

docs_encontrados = vectorstore.similarity_search(query, k=3)

print(f"\n Documentos encontrados: {len(docs_encontrados)}")
print("\n--- Resultado más relevante ---")
print(docs_encontrados[0].page_content[:500])
print("...")

print("\n Sistema RAG funcionando correctamente")

Instalando modelo de embeddings...
Creando embeddings con modelo local...
Modelo de embeddings cargado (funciona offline y sin cuotas)
Creando vector store con FAISS...
Vector store creado con éxito
   - Total de vectores: 30

PRUEBA DE BÚSQUEDA SEMÁNTICA

 Query: ¿Qué playas hay en el norte de Tenerife?

 Documentos encontrados: 3

--- Resultado más relevante ---
tendréis poca playa (especialmente a la del Ancón) y tened mucho respeto al mar en 
estas playas, que la corriente es muy traicionera. 
 
• Puerto de La Cruz 
Es la zona turística del Norte de Tenerife. De hecho, hace años era el núcleo turístico 
de la isla hasta que empezó a ganar mucho peso el Sur de Tenerife. 
 
 
Para visitar el Puerto de la Cruz, os recomendaría aparcar en la explanada del muelle 
e ir caminando desde el Muelle hasta la Playa Martiánez recorriendo la Calle de 
Santo Doming
...

 Sistema RAG funcionando correctamente


In [17]:
# Configurar el modelo de lenguaje (Gemini 2.0 Flash)
print("🤖 Configurando modelo Gemini 2.0 Flash...")

# Parámetros del modelo
MODEL_NAME = "gemini-2.5-flash"
TEMPERATURE = 0.7
MAX_TOKENS = 2000

llm = ChatGoogleGenerativeAI(
    model=MODEL_NAME,
    temperature=TEMPERATURE,
    max_output_tokens=MAX_TOKENS,
    google_api_key=GOOGLE_API_KEY
)

print(f"Modelo configurado: {MODEL_NAME}")
print(f"   - Temperatura: {TEMPERATURE}")
print(f"   - Max tokens: {MAX_TOKENS}")

# Prueba rápida del modelo
print("\n" + "="*60)
print("🧪 PRUEBA DEL MODELO")
print("="*60)

try:
    test_response = llm.invoke("Di 'Hola, soy Gemini 2.5 Flash' en una frase corta")
    print(f"\nRespuesta del modelo: {test_response.content}")
    print("Modelo funcionando correctamente")
except Exception as e:
    print(f"Error al probar el modelo: {e}")

🤖 Configurando modelo Gemini 2.0 Flash...
Modelo configurado: gemini-2.5-flash
   - Temperatura: 0.7
   - Max tokens: 2000

🧪 PRUEBA DEL MODELO

Respuesta del modelo: Hola, soy Gemini 2.5 Flash.
Modelo funcionando correctamente


In [18]:
# Sistema de memoria conversacional manual
class ConversationHistory:
    """Gestiona el historial de conversación con límite de tokens"""

    def __init__(self, max_interactions=5):
        self.history = []
        self.max_interactions = max_interactions

    def add_user_message(self, message):
        """Agregar mensaje del usuario"""
        self.history.append({"role": "user", "content": message})
        self._trim_history()

    def add_assistant_message(self, message):
        """Agregar mensaje del asistente"""
        self.history.append({"role": "assistant", "content": message})
        self._trim_history()

    def _trim_history(self):
        """Mantener solo las últimas N interacciones"""
        # Cada interacción = 1 user + 1 assistant (2 mensajes)
        max_messages = self.max_interactions * 2
        if len(self.history) > max_messages:
            self.history = self.history[-max_messages:]

    def get_formatted_history(self):
        """Obtener historial formateado para el prompt"""
        if not self.history:
            return ""

        formatted = "\n\nHistorial de conversación:\n"
        for msg in self.history:
            role = "Usuario" if msg["role"] == "user" else "Asistente"
            formatted += f"{role}: {msg['content']}\n"
        return formatted

    def clear(self):
        """Limpiar historial"""
        self.history = []

    def get_context_summary(self):
        """Obtener resumen del contexto actual"""
        return f"Mensajes en memoria: {len(self.history)} ({len(self.history)//2} interacciones)"

# Crear instancia de memoria
conversation_memory = ConversationHistory(max_interactions=5)

print("Sistema de memoria conversacional creado")
print(f"   - Capacidad: {conversation_memory.max_interactions} interacciones")
print(f"   - {conversation_memory.get_context_summary()}")

Sistema de memoria conversacional creado
   - Capacidad: 5 interacciones
   - Mensajes en memoria: 0 (0 interacciones)


In [19]:
# Función principal del asistente turístico
def asistente_turistico(pregunta_usuario, use_rag=True, verbose=True):
    """
    Asistente turístico con RAG y function calling

    Args:
        pregunta_usuario: Pregunta del usuario
        use_rag: Si usar búsqueda en la guía turística
        verbose: Mostrar información de debug

    Returns:
        str: Respuesta del asistente
    """

    if verbose:
        print("\n" + "="*70)
        print(f" Usuario: {pregunta_usuario}")
        print("="*70)

    # 1. Detectar si necesita información del clima
    needs_weather = any(palabra in pregunta_usuario.lower()
                       for palabra in ['clima', 'tiempo', 'temperatura', 'llover', 'lluvia', 'sol'])

    weather_info = ""
    if needs_weather:
        if verbose:
            print("\n  Detectada consulta sobre clima...")

        # Extraer fecha y ubicación (simplificado)
        fecha_hoy = datetime.now().strftime('%Y-%m-%d')
        ubicacion = "Tenerife"

        # Detectar ubicación específica
        if "norte" in pregunta_usuario.lower():
            ubicacion = "Norte de Tenerife"
        elif "sur" in pregunta_usuario.lower():
            ubicacion = "Sur de Tenerife"
        elif "teide" in pregunta_usuario.lower():
            ubicacion = "Teide"

        # Llamar a la función del clima
        weather_data = get_weather(fecha_hoy, ubicacion)

        if weather_data["status"] == "success":
            weather_info = f"\n\nInformación del clima para {weather_data['ubicacion']}:\n"
            weather_info += f"- Temperatura: {weather_data['temperatura']}\n"
            weather_info += f"- Condiciones: {weather_data['descripcion']}\n"
            weather_info += f"- Humedad: {weather_data['humedad']}\n"
            weather_info += f"- Probabilidad de lluvia: {weather_data['probabilidad_lluvia']}\n"
            weather_info += f"- Recomendación: {weather_data['recomendacion']}\n"
        else:
            weather_info = f"\n\n No se pudo obtener información del clima: {weather_data['error']}\n"

    # 2. Búsqueda RAG en la guía turística
    context = ""
    sources = []

    if use_rag:
        if verbose:
            print(" Buscando en la guía turística...")

        relevant_docs = vectorstore.similarity_search(pregunta_usuario, k=3)

        if relevant_docs:
            context = "\n\nInformación de la guía turística de Tenerife:\n"
            for i, doc in enumerate(relevant_docs, 1):
                context += f"\n[Fuente {i}]\n{doc.page_content}\n"
                sources.append(f"Página {doc.metadata.get('page', 'N/A')}")

            if verbose:
                print(f"   Encontrados {len(relevant_docs)} documentos relevantes")

    # 3. Construir el prompt completo
    system_prompt = """Eres un asistente turístico experto en Tenerife. Tu trabajo es ayudar a los turistas a planificar su visita.

INSTRUCCIONES:
- Proporciona información precisa y útil basada en la guía turística
- Si tienes información del clima, inclúyela en tu respuesta
- Sé amigable, conversacional y entusiasta
- Mantén respuestas concisas pero informativas
- Si no tienes información específica, sé honesto y sugiere alternativas
- IMPORTANTE: Cita las fuentes cuando uses información de la guía

CONTEXTO DISPONIBLE:
"""

    # Añadir historial de conversación
    history_text = conversation_memory.get_formatted_history()
    if history_text:
        system_prompt += history_text

    # Añadir contexto de la guía
    if context:
        system_prompt += context

    # Añadir información del clima
    if weather_info:
        system_prompt += weather_info

    # 4. Construir el mensaje del usuario
    user_message = f"\nPregunta actual: {pregunta_usuario}\n\nResponde de forma amigable y útil."

    # 5. Llamar al modelo
    if verbose:
        print("Generando respuesta...")

    full_prompt = system_prompt + user_message
    response = llm.invoke(full_prompt)
    respuesta = response.content

    # 6. Actualizar memoria
    conversation_memory.add_user_message(pregunta_usuario)
    conversation_memory.add_assistant_message(respuesta)

    # 7. Mostrar respuesta
    if verbose:
        print("\n Asistente:")
        print("-" * 70)

    print(respuesta)

    if verbose and sources:
        print("\n Fuentes consultadas:", ", ".join(sources))
        print(f" {conversation_memory.get_context_summary()}")

    return respuesta

print(" Asistente turístico configurado y listo")

 Asistente turístico configurado y listo


In [20]:
# Ejemplos de conversación
print("="*70)
print(" EJEMPLOS DE USO DEL ASISTENTE")
print("="*70)

# Ejemplo 1: Pregunta sobre lugares
print("\n EJEMPLO 1: Lugares turísticos")
asistente_turistico("¿Qué lugares me recomiendas visitar en el norte de Tenerife?")

# Ejemplo 2: Pregunta con clima
print("\n\n EJEMPLO 2: Clima")
asistente_turistico("¿Qué tiempo hace hoy en el sur? ¿Es buen día para la playa?")

# Ejemplo 3: Seguimiento (usa memoria)
print("\n\n EJEMPLO 3: Seguimiento conversacional")
asistente_turistico("¿Y cómo llego allí?")

 EJEMPLOS DE USO DEL ASISTENTE

 EJEMPLO 1: Lugares turísticos

 Usuario: ¿Qué lugares me recomiendas visitar en el norte de Tenerife?
 Buscando en la guía turística...
   Encontrados 3 documentos relevantes
Generando respuesta...

 Asistente:
----------------------------------------------------------------------
¡Hola! ¡Qué ganas de que descubras la maravillosa zona norte de Tenerife! Es una parte de la isla con un encanto muy especial. Aquí tienes un par de recomendaciones que no te puedes perder:

1.  **El Teide y su Parque Nacional:** ¡Es una visita obligatoria! Como dice la guía, si vas a Tenerife y no subes al Teide, "simplemente no habéis estado en Tenerife" [Fuente 1].
    *   Te sugiero subir por la carretera TF24 (carretera de La Esperanza) desde La Laguna.
    *   Haz una parada en el **Mirador de La Tarta**; su nombre viene de los colores de la piedra que parecen una tarta, ¡y si tienes suerte y está nublado en el Norte, verás un mar de nubes espectacular! También puedes di

'¡Claro que sí! Entiendo que quieres saber cómo llegar a esas maravillosas playas del sur. 🎉\n\nDado que la guía no especifica cómo llegar a las playas del sur en general, te puedo dar algunas indicaciones útiles para moverte por la isla:\n\n*   **En coche de alquiler:** Es la forma más cómoda y flexible de explorar Tenerife. Podrás llegar a cualquier playa del sur con facilidad y a tu propio ritmo. Las principales carreteras que conectan el norte y el sur son la TF-1 (autovía del sur) y la TF-5 (autovía del norte), y luego tendrías que tomar las salidas adecuadas hacia las zonas costeras del sur.\n*   **En autobús (guagua):** La compañía Titsa opera una excelente red de autobuses por toda la isla. Puedes consultar sus rutas y horarios en su página web para encontrar la línea que te lleve a la zona del sur que te interese.\n\nPara darte una ruta más precisa, ¿me podrías decir desde dónde sales o a qué playa específica del sur te gustaría ir? ¡Así te puedo ayudar mejor a planificar tu c

In [21]:
# Celda 10: Chat interactivo
def chat_interactivo():
    """Modo chat interactivo con el asistente"""
    print("\n" + "="*70)
    print("  ASISTENTE TURÍSTICO DE TENERIFE - MODO CHAT")
    print("="*70)
    print("\nComandos especiales:")
    print("  - 'salir' o 'exit': Terminar el chat")
    print("  - 'limpiar' o 'clear': Borrar historial")
    print("  - 'historial': Ver conversación actual")
    print("\n" + "="*70 + "\n")

    while True:
        try:
            # Obtener input del usuario
            pregunta = input(" Tú: ").strip()

            # Comandos especiales
            if pregunta.lower() in ['salir', 'exit', 'quit']:
                print("\n ¡Hasta pronto! Disfruta tu visita a Tenerife.")
                break

            if pregunta.lower() in ['limpiar', 'clear']:
                conversation_memory.clear()
                print(" Historial borrado.\n")
                continue

            if pregunta.lower() in ['historial', 'history']:
                print("\n Historial de conversación:")
                print(conversation_memory.get_formatted_history())
                print()
                continue

            if not pregunta:
                continue

            # Procesar pregunta
            asistente_turistico(pregunta, verbose=False)
            print()

        except KeyboardInterrupt:
            print("\n\n Chat interrumpido. ¡Hasta pronto!")
            break
        except Exception as e:
            print(f"\n Error: {str(e)}\n")

print(" Modo chat interactivo disponible")
print("Ejecuta: chat_interactivo()")

 Modo chat interactivo disponible
Ejecuta: chat_interactivo()


In [22]:
#Chat interactivo
chat_interactivo()


  ASISTENTE TURÍSTICO DE TENERIFE - MODO CHAT

Comandos especiales:
  - 'salir' o 'exit': Terminar el chat
  - 'limpiar' o 'clear': Borrar historial
  - 'historial': Ver conversación actual


 Tú: Hola, buenos dias
¡Hola, muy buenos días! ¡Me alegra saludarte!

Estoy aquí para ayudarte a planificar tu aventura en la maravillosa Tenerife. ¿Hay algo en particular en lo que te pueda ayudar hoy o alguna duda que tengas sobre la isla? ¡Estoy listo para compartir contigo los mejores consejos y recomendaciones! 😊

 Tú: Me llamo iván, que playas me recomiendas visitar?
¡Hola, Iván! ¡Qué alegría que me preguntes por las playas! Tenerife tiene algunas calas y arenales que son auténticos tesoros. ¡Te voy a dar unas recomendaciones que te encantarán!

Aquí tienes algunas playas que la guía destaca y que podrían ser perfectas para ti:

1.  **Playa de Benijo:** ¡Esta playa es una maravilla! Es una playa de arena negra que, si el día está despejado, te regalará uno de los atardeceres más espectacula